In [1]:
# Install library for reading JPK File
%pip uninstall pyafmreader --yes
%pip install git+https://github.com/jlopezalo/pyafmreader.git

Found existing installation: pyafmreader x.x.x
Uninstalling pyafmreader-x.x.x:
  Successfully uninstalled pyafmreader-x.x.x
Note: you may need to restart the kernel to use updated packages.
  Cloning https://github.com/jlopezalo/pyafmreader.git to /private/var/folders/4k/ps22fmxj3gx6vls78t2p63wh0000gn/T/pip-req-build-2my21xni
  Running command git clone --filter=blob:none --quiet https://github.com/jlopezalo/pyafmreader.git /private/var/folders/4k/ps22fmxj3gx6vls78t2p63wh0000gn/T/pip-req-build-2my21xni
  Resolved https://github.com/jlopezalo/pyafmreader.git to commit 455c83796458190d25d9ff9b6a4ad8b3e40680b4
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/TkTech/fasterzip.git to /private/var/folders/4k/ps22fmxj3gx6vls78t2p63wh0000gn/T/pip-install-a5nlt_lg/fasterzip_29a072a04d39430e9cfcb88d7014a389
  Running command git clone --filter=blob:none --quiet https://github.com/TkTech/fasterzip.git /private/var/folders/4k/ps22fmxj3gx6vls78t2p63wh0000gn/T/pip-install-a5nlt_

In [2]:
# Install library for AFM data Rheology Analysis
%pip uninstall pyafmrheo --yes
%pip install git+https://github.com/jlopezalo/pyafmrheo.git

Found existing installation: pyafmrheo 0.0.1
Uninstalling pyafmrheo-0.0.1:
  Successfully uninstalled pyafmrheo-0.0.1
Note: you may need to restart the kernel to use updated packages.
  Cloning https://github.com/jlopezalo/pyafmrheo.git to /private/var/folders/4k/ps22fmxj3gx6vls78t2p63wh0000gn/T/pip-req-build-4unmdrbw
  Running command git clone --filter=blob:none --quiet https://github.com/jlopezalo/pyafmrheo.git /private/var/folders/4k/ps22fmxj3gx6vls78t2p63wh0000gn/T/pip-req-build-4unmdrbw
  Resolved https://github.com/jlopezalo/pyafmrheo.git to commit 43295b580a50f00097694a503bc5f35d11e24c98
  Preparing metadata (setup.py) ... done
  Created wheel for pyafmrheo: filename=pyafmrheo-0.0.1-py3-none-any.whl size=33274 sha256=c921ede2050eea63949872effa441b85032ea622cc9c9bfd5e4616503061977b
  Stored in directory: /private/var/folders/4k/ps22fmxj3gx6vls78t2p63wh0000gn/T/pip-ephem-wheel-cache-zesl290v/wheels/45/a3/5e/2d19803e3cbf5fe45f904090127b31ec30fc304c6b6e5e0f70
Successfully built pya

In [3]:
# Import libraries we will need
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Get file reader from library
from pyafmreader import loadfile

# Get data analysis tools
from pyafmrheo.routines.TingFit import doTingFit

In [4]:
# Define global variables
file_path = '/Users/javierlopez/Documents/Datasets/05-11-2021/E/map-data-2021.11.05-15.41.45.699.jpk-force-map'
# Shapes available: paraboloid, pyramid
indenter_shape = "paraboloid"
tip_parameter = 30 * 1e-9 # meters
# Poisson ratio
poisson_ratio = 0.5
# Max non contact region
maxnoncontact = 2.5 * 1e-6
# Window to find cp
windowforCP = 350e-9
# Smooth window
smooth_w = 1
# t0 scaling factor
t0 = 1
# Viscous drag for PFQNM
vdrag = 0.77*1e-6
# If None it will use the deflection sensitivity from the file
deflection_sensitivity = None # m/V
# If None it will use the spring constant from the file
spring_constant = None # N/m
# General plotting params
plt.rcParams["figure.figsize"] = (10,5)

In [5]:
file = loadfile(file_path)

In [6]:
filemetadata = file.filemetadata
print(filemetadata['file_type'])

.jpk-force-map


In [7]:
closed_loop = filemetadata['z_closed_loop']
file_deflection_sensitivity = filemetadata['defl_sens_nmbyV'] #nm/V
file_spring_constant = filemetadata['spring_const_Nbym'] #N/m
height_channel = filemetadata['height_channel_key']

if not deflection_sensitivity: deflection_sensitivity = file_deflection_sensitivity / 1e9 #m/V
if not spring_constant: spring_constant = file_spring_constant

In [8]:
param_dict = {
    'def_sens': deflection_sensitivity,
    'k': spring_constant,
    'height_channel': height_channel,
    'poc_win': windowforCP,
    'curve_seg': 'extend',
    'fit_range_type': 'indentation',
    'min_ind': -np.inf,
    'max_ind': np.inf,
    'fit_line': False,
    'd0': 0.0,
    'f0': 0.0,
    'auto_init_E0': True,
    'correct_tilt': True,
    'tilt_max_offset': 1e-6,
    'tilt_min_offset': 10e-9,
    'vdragcorr': False,
    'polyordr': None,
    'rampspeed': None,
    'compute_v_flag': False,
    'contact_offset': maxnoncontact,
    'pts_downsample': 300,
    'vdrag': vdrag,
    'auto_init_betaE': True,
    'fluid_exp': None,
    'contact_model': indenter_shape,
    'tip_param': tip_parameter,
    'model_type': 'analytical',
    't0': t0,
    'smoothing_win': smooth_w
}

In [9]:
curve_idx = 0
force_curve = file.getcurve(curve_idx)
force_curve.preprocess_force_curve(param_dict['def_sens'], param_dict['height_channel'])
if filemetadata['file_type'] in ('.jpk-force', '.jpk-force-map', '.jpk-qi-data'):
    force_curve.shift_height()

In [10]:
ting_result, hertz_result = doTingFit(force_curve, param_dict)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pyafmrheo/models/hertz.py:161: RuntimeWarning: divide by zero encountered in true_divide
  a = (self.get_residuals(indentation, force, sample_height)**2/force)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pyafmrheo/models/ting.py:265: RuntimeWarning: divide by zero encountered in true_divide
  a = (self.get_residuals(time, F, delta, t0, idx_tm, smooth_w, v0t, v0r)**2/F)


In [11]:
hertz_result.fit_report()


        # Fit parameters
        Indenter shape: paraboloid

        Tip paraneter: 3.0000000000000004e-08

        BEC Model: None

        Number of free parameters: 3

        delta0: 7.853249485014762e-08

        E0: 1221.941332533732

        f0: 2.8144553067024763e-12

        slope: None

        # Fit metrics
        MAE: -2.9078232494410534e-18

        MSE: 1.1008181820463074e-22

        RMSE: 1.0491988286527522e-11

        Rsq: 0.9951614367839628

        Chisq: -4.495450714407658e-09

        RedChisq: -1.4984835714692193e-09

        


In [12]:
ting_result.fit_report()


        # Fit parameters
        Indenter shape: paraboloid

        Tip paraneter: 3.0000000000000004e-08

        Model Format: analytical

        Viscous Drag: 0

        Smooth Window: 1

        t0: 1

        Maximum Indentation Time: 150

        Number of free parameters: 4

        E0: 851.1371869655261

        tc: 0.005533169684531096

        betaE: 0.16165034640081477

        F0: 2.9276233748293746e-11

        # Fit metrics
        MAE: 2.957600606354558e-11

        MSE: 1.6532218058968172e-21

        RMSE: 4.0659830372209094e-11

        Rsq: 0.961464744762088

        Chisq: -9.368401976085534e-09

        RedChisq: -2.3421004940213835e-09

        
